In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import time

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [3]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_01-03/'               # Harmon server
name_project

'project_repartition_v4.0/output_repar_v9.2_01-01/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
dir_stack = dir_project + info_basic['dir_stack']
dir_ds = dir_project + info_basic['dir_ds']
dir_partition = dir_project + info_basic['dir_partition']
dir_image = dir_project + info_basic['dir_image'] + 'regular_FJ/'
if os.path.exists(dir_image) == False:
    os.makedirs(dir_image)

In [7]:
key_subworks = info_basic['key_subworks']
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '11--19-03',
 '12--20-03',
 '13--21-03',
 '14--22-03',
 '15--23-03',
 '16--24-03',
 '17--25-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '24--32-03',
 '25--33-03',
 '26--34-03',
 '27--35-03',
 '28--36-03',
 '29--37-03',
 '30--41-03',
 '31--42-03',
 '32--43-03',
 '33--46-03',
 '34--47-03',
 '35--48-03',
 '36--52-03',
 '37--53-03',
 '38--07-04',
 '39--08-04',
 '40--09-04',
 '41--10-04',
 '42--12-04',
 '43--13-04',
 '44--17-04',
 '45--20-04',
 '46--21-04',
 '47--22-04',
 '48--23-04',
 '49--24-04',
 '50--25-04',
 '51--27-04',
 '52--28-04',
 '53--29-04',
 '54--30-04',
 '55--31-04',
 '56--32-04',
 '57--33-04',
 '58--34-04',
 '59--35-04',
 '60--36-04',
 '61--41-04',
 '62--42-04',
 '63--47-04',
 '64--48-04',
 '65--49-04',
 '66--52-04',
 '67--53-04',
 '68--08-05',
 '69--12-05',
 '70--13-05',
 '71--14-05',
 '72--16-05',
 

In [8]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [9]:
stations = {}
lat = {}
lon = {}
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations[key] = stations_this
    lat[key] = lat_stations_this.astype(float)
    lon[key] = lon_stations_this.astype(float)

In [10]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [11]:
#%%capture
d_len = 31
c_map = 'jet'
v_min = 0.1
v_max = None
info_basic['v_min'] = v_min
info_basic['v_max'] = v_max
info_basic['c_map'] = c_map
#t0 = info_basic['t0']
#v_tag = info_basic['v_tag']
for key_subwork in key_subworks:
    print(key_subwork)
    fig,ax = plt.subplots(3,3,figsize= (22,18))
    # settings for subwork
    
    
    #start = info_basic_bi['start'][key_subwork]
    #interval = info_basic_bi['interval'][key_subwork]
    #flag_time = info_basic_bi['flag_time'][key_subwork]
    start = info_basic['start']
    interval = info_basic['interval']
    flag_time = info_basic['flag_time']

    xlim_f = [1,15]
    xlim_T = [-1,1]

    # plot area
    
    stalist = stations[key_subwork]
    lat_this = lat[key_subwork]
    lon_this = lon[key_subwork]
    nsta = len(stalist)
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)

    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon_this,lat_this)
    for i in range(len(faults)):
        ax[0][0].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
    

    # plot ncfs and ncfst
    # linear stack
    
    outname = str(key_subwork)+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs'][:]
    r = ncffile['r'][:]
    ncffile.close()
    # timewindow-filtered stack
    outname = str(key_subwork)+'_gather_timewindow.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_remove = ncffile['ncfs'][:]
    ncffile.close()

    f = info_basic_bi['f']
    t = info_basic_bi['t']
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    ncfst_remove = mathlib.freq_time(ncfs_sum_remove)
    
    title1 = "Linear Stack subarea frequency for "+str(key_subwork)
    #xlim_f = [0,15]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f,ncfs_sum_linear,r,title1,xlim_f,0)
    ax[0][2] = plotlib.plot_ncfs(ax[0][2],f,ncfs_sum_remove,r,title1,xlim_f,0)
    
    # plot fj
    ds = h5py.File(dir_ds+'ds_'+str(key_subwork)+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    ds_remove = ds['ds_remove'][0]
    ds_linear = plotlib.smooth_ds(ds_linear)
    ds_remove = plotlib.smooth_ds(ds_remove)
    ds.close()
    
    c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
    #c = np.linspace(100,2000,1000)
    
    title0 = "Linear Stack subarea time for "+str(key_subwork)
    ax[1][0] = plotlib.plot_ncfst(ax[1][0],t,ncfst_linear[start::interval],r[start::interval],title0,flag_time,xlim_T,0)
    title0 = "Remove Stack subarea time for "+str(key_subwork)
    ax[1][1] = plotlib.plot_ncfst(ax[1][1],t,ncfst_remove[start::interval],r[start::interval],title0,flag_time,xlim_T,0)
    for i in range(len(r)):
        ax[1][1].plot()
    
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[2][0] = plotlib.plot_fj(ax[2][0],ds_linear,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
    ax[2][0].set_xlim(xlim_f)

    title0 = "Remove stack dispersion curve "+str(d_len)+' days'
    ax[2][1] = plotlib.plot_fj(ax[2][1],ds_remove,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
    ax[2][1].set_xlim(xlim_f)


    plt.tight_layout()
    plt.savefig(dir_image+str(key_subwork)+'_compare_remove.png',dpi=100)
    plt.close()
    #time.sleep(7)

1--08-03
2--09-03
3--10-03
4--11-03
5--12-03
6--13-03
7--14-03
8--15-03
9--16-03
10--18-03
11--19-03
12--20-03
13--21-03
14--22-03
15--23-03
16--24-03
17--25-03
18--26-03
19--27-03
20--28-03
21--29-03
22--30-03
23--31-03
24--32-03
25--33-03
26--34-03
27--35-03
28--36-03
29--37-03
30--41-03
31--42-03
32--43-03
33--46-03
34--47-03
35--48-03
36--52-03
37--53-03
38--07-04
39--08-04
40--09-04
41--10-04
42--12-04
43--13-04
44--17-04
45--20-04
46--21-04
47--22-04
48--23-04
49--24-04
50--25-04
51--27-04
52--28-04
53--29-04
54--30-04
55--31-04
56--32-04
57--33-04
58--34-04
59--35-04
60--36-04
61--41-04
62--42-04
63--47-04
64--48-04
65--49-04
66--52-04
67--53-04
68--08-05
69--12-05
70--13-05
71--14-05
72--16-05
73--17-05
74--19-05
75--22-05
76--24-05
77--26-05
78--27-05
79--28-05
80--29-05
81--30-05
82--32-05
83--33-05
84--35-05
85--36-05
86--37-05
87--38-05
88--39-05
89--40-05
90--41-05
91--42-05
92--43-05
93--44-05
94--48-05
95--49-05
96--51-05
97--52-05
98--53-05
99--54-05
100--07-06
101--08-

In [12]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)